In [0]:
#Install Tensor flow
!pip install -q tensorflow==2.0.beta1

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

In [0]:
#Basic Time Series with Sine wave with no noise.
#series = np.sin(0.1*np.arange(200)) 
#Basic Time Series with Sine wave with no noise.
series = np.sin(0.1*np.arange(200)) + np.random.randn(200)*0.1

#plot it
plt.plot(series);

In [0]:
###Build the data set
# let's see if we can use T past values to predict the next value
T=10
X=[]
Y=[]

#Final Target Index = Final t + T = [len(series)-T-1]+T = len(series)-1
for t in range(len(series)-T):
  x= series[t:t+T]
  X.append(x)
  y = series[t+T]
  Y.append(y)

X = np.array(X).reshape(-1, T)
Y = np.array(Y)
N=len(X)
print("X.Shape:", X.shape, "Y.Shape:", Y.shape)

In [0]:
## Try Autoregressive linear model
i = Input(shape=(T,))
x = Dense(1)(i)
model = Model(i, x)
model.compile(loss='mse', optimizer=Adam(lr=0.1),)

#Train the RNN
r = model.fit(
    X[:-N//2], Y[:-N//2], 
    epochs=80, 
    validation_data=(X[-N//2:], Y[-N//2:]),
)

In [0]:
# Plot loss per iteration
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [0]:
# "Wrong" forecast using true targets

validation_target = Y[-N//2:]
validation_predictions = []

# index of first validation input
i = -N//2

while len(validation_predictions) < len(validation_target):
  p = model.predict(X[i].reshape(1, -1))[0,0] # 1x1 array -> scalar
  i += 1
  
  # update the predictions list
  validation_predictions.append(p)

In [0]:
plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

In [0]:
# Forecast future values (use only self-predictions for making future predictions)

validation_target = Y[-N//2:]
validation_predictions = []

# first validation input
last_x = X[-N//2] # 1-D array of length T

while len(validation_predictions) < len(validation_target):
  p = model.predict(last_x.reshape(1, -1))[0,0] # 1x1 array -> scalar
  
  # update the predictions list
  validation_predictions.append(p)
  
  # make the new input
  #roll = shift function i.e. use last prediction value for future prediction
  last_x = np.roll(last_x, -1)
  last_x[-1] = p

In [0]:
plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()